In [4]:
# Libraries and dependencies
import os
import alpaca_trade_api as tradeapi
import pandas as pd 

from dotenv import load_dotenv

%matplotlib inline

In [5]:
# Loading .env 
load_dotenv()

True

In [ ]:
# Set Alpaca API key/ secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Conection
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")